In [1]:
!pip install transformers
!pip install av


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 21.3 MB/s eta 0:00:00


In [2]:
from google.colab import drive
from transformers import ViTImageProcessor, ViTForImageClassification
from transformers import VideoMAEImageProcessor, VideoMAEForPreTraining
from torch import Tensor
import torch
from PIL import Image
import pandas as pd
import os
from transformers import AutoImageProcessor, ViTModel, VideoMAEModel
import av
import numpy as np
import csv
import pandas as pd


In [3]:
drive.mount('/content/drive')

imgs_path="/content/drive/MyDrive/TFM/Embeddings/ImgEmbeddings"

Mounted at /content/drive


In [4]:
img_processor = AutoImageProcessor.from_pretrained('google/vit-base-patch16-224')
img_model = ViTModel.from_pretrained('google/vit-base-patch16-224')

video_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
video_model = VideoMAEModel.from_pretrained("MCG-NJU/videomae-base")

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [5]:
def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

In [6]:
#example videos 64784974967e4199e327074d

In [7]:
def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices

In [8]:
base_folder = "/content/drive/MyDrive/TFM/Embeddings/ImgEmbeddings/Images"
with open('/content/drive/MyDrive/TFM/Embeddings/ImgEmbeddings/ImgEmbeddings.csv', 'a') as file:
  for folder_name in os.listdir(base_folder):
    #folder_name= '64784974967e4199e327074d' #Added
    folder_path = os.path.join(base_folder, folder_name)

    if os.path.isdir(folder_path):
      image_arrays = []
      emb_array=[]
      img_ids = [folder_name]


      for image_name in os.listdir(folder_path):
        #print(image_name)
        image_path = os.path.join(folder_path, image_name)
        #img_ids.append()
        if os.path.isfile(image_path) and image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            img = Image.open(image_path)

            image_arrays.append(img)
      if image_arrays != []:
        inputs = img_processor(images=image_arrays, return_tensors="pt")
        with torch.no_grad():
          outputs = img_model(**inputs).pooler_output.detach().numpy().flatten().tolist()

        emb_array = emb_array + outputs

      if os.path.isfile(image_path) and image_name.lower().endswith(('.mp4')):
        try:
          container = av.open(image_path)
          indices = sample_frame_indices(clip_len=16, frame_sample_rate=4, seg_len=container.streams.video[0].frames)
          video = read_video_pyav(container, indices)
          inputs = video_processor(list(video), return_tensors="pt")
          outputs = video_model(**inputs).last_hidden_state.detach().numpy().flatten().tolist()

          emb_array = emb_array + outputs
        except:
          pass
      new_row = img_ids + emb_array

      writer = csv.writer(file)
      writer.writerow(new_row)



/usr/local/lib/python3.10/dist-packages/transformers/feature_extraction_utils.py:149: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  return torch.tensor(value)


In [9]:
data = []
with open('/content/drive/MyDrive/TFM/Embeddings/ImgEmbeddings/ImgEmbeddings.csv', 'r') as csv_file:
    for line in csv_file:
        data.append(line.strip().split(',')[0])

In [10]:
print(len(data))
print(len(set(data)))

5865
5865
